# Laboratorio 6
Francisco Castillo - 21562

Diego Lemus - 21

## Task 1
### ¿Qué es Prioritized Sweeping para ambientes determinísticos?
Es una técnica de planeamiento que permite acelerar la propagación de los valores de los estados sin necesidad de actualizarlos todos uniformemente. En lugar de recorrer cada estado del espacio, se utiliza una cola de prioridades en la que se ordenan los estados según la magnitud del cambio que producen en sus valores. Cuando un estado se actualiza significativamente, los estados predecesores que llevan a él se incorporan a la cola para ser actualizados después. De este modo, los cambios en el valor de un estado se propagan hacia atrás de manera más eficiente, evitando cálculos innecesarios y enfocándose en las partes del espacio de estados donde realmente importa. En ambientes determinísticos esto resulta especialmente potente, ya que cada acción tiene un único resultado y la propagación de los valores puede seguir trayectorias claras.

### ¿Qué es Trajectory Sampling?
Es una técnica que evita la necesidad de explorar todo el árbol de estados y acciones posibles durante el planeamiento. En lugar de un análisis exhaustivo, el método simula trayectorias completas o episodios ficticios siguiendo la política actual y va actualizando los valores de los estados y acciones visitados en ese recorrido.

### ¿Qué es Upper Confidence Bounds para Árboles (UCT)?
Es una estrategia de selección que resuelve el dilema entre exploración y explotación. Para cada acción, combina el valor promedio estimado de los resultados obtenidos hasta el momento con un término de confianza que favorece aquellas opciones que han sido probadas pocas veces. De esta manera, el algoritmo asegura que no se quede únicamente con las ramas que parecen mejores al inicio, sino que también explore alternativas menos visitadas que podrían resultar óptimas.

## Task 2. MCTS

In [ ]:
import gymnasium as gym
import numpy as np
import math
import random

class Node:
    def __init__(self, state, parent=None, parent_action=None):
        self.state = state
        self.parent = parent
        self.parent_action = parent_action
        self.children = {}
        self.visits = 0
        self.total_value = 0.0

    def add_child(self, action, child_node):
        self.children[action] = child_node

    def is_fully_expanded(self, action_space_size):
        return len(self.children) == action_space_size

    def uct_value(self, c_param):
        if self.visits == 0:
            return float('inf')
        return (self.total_value / self.visits) + c_param * math.sqrt(2 * math.log(self.parent.visits) / self.visits)

class MCTS:
    def __init__(self, env, c_param=1.0):
        self.env = env
        self.c_param = c_param
        self.root = None

    def select(self):
        current_node = self.root
        while current_node.is_fully_expanded(self.env.action_space.n):
            best_child = None
            best_uct_value = -float('inf')
            for action, child in current_node.children.items():
                uct_val = child.uct_value(self.c_param)
                if uct_val > best_uct_value:
                    best_uct_value = uct_val
                    best_child = child
            if best_child is None:
                 break
            current_node = best_child
            if self.env.unwrapped.s in self.env.unwrapped.desc.flatten().tolist() and self.env.unwrapped.desc.flatten()[self.env.unwrapped.s] in [b'H', b'G']:
                break

        return current_node

    def expand(self, node):
        if node.state is None:
             return None

        possible_actions = list(range(self.env.action_space.n))
        random.shuffle(possible_actions)

        for action in possible_actions:
            if action not in node.children:
                original_state = self.env.unwrapped.s
                self.env.unwrapped.s = node.state

                next_state, reward, terminated, truncated, info = self.env.step(action)

                self.env.unwrapped.s = original_state

                new_node = Node(next_state, parent=node, parent_action=action)
                node.add_child(action, new_node)
                return new_node

        return None

    def simulate(self, node):
        original_state = self.env.unwrapped.s
        self.env.unwrapped.s = node.state

        terminated = False
        truncated = False
        total_reward = 0

        while not terminated and not truncated:
            action = self.env.action_space.sample()
            next_state, reward, terminated, truncated, info = self.env.step(action)
            total_reward += reward
            if not terminated and not truncated:
                self.env.unwrapped.s = next_state


        self.env.unwrapped.s = original_state

        return total_reward

    def backpropagate(self, node, reward):
        current_node = node
        while current_node is not None:
            current_node.visits += 1
            current_node.total_value += reward
            current_node = current_node.parent

    def run_mcts(self, num_simulations):
        if self.root is None:
            return

        for _ in range(num_simulations):
            leaf_node = self.select()

            if self.env.unwrapped.s in self.env.unwrapped.desc.flatten().tolist() and self.env.unwrapped.desc.flatten()[self.env.unwrapped.s] in [b'H', b'G']:
                original_state = self.env.unwrapped.s
                self.env.unwrapped.s = leaf_node.state
                reward = 0
                if self.env.unwrapped.desc.flatten()[self.env.unwrapped.s] == b'G':
                     reward = 1.0
                self.env.unwrapped.s = original_state
                self.backpropagate(leaf_node, reward)
                continue

            new_node = self.expand(leaf_node)

            if new_node is None:
                simulation_reward = self.simulate(leaf_node)
                self.backpropagate(leaf_node, simulation_reward)
            else:
                simulation_reward = self.simulate(new_node)
                self.backpropagate(new_node, simulation_reward)

    def get_best_action(self):
        if self.root is None or not self.root.children:
            return None

        best_action = None
        best_visits = -1

        for action, child in self.root.children.items():
            if child.visits > best_visits:
                best_visits = child.visits
                best_action = action

        return best_action

env = gym.make("FrozenLake-v1", is_slippery=True)
num_episodes = 100
num_simulations_per_step = 5000

episode_rewards = []
successes = 0

for episode in range(num_episodes):
    observation, info = env.reset()
    mcts = MCTS(env, c_param=1.0)
    total_episode_reward = 0
    terminated = False
    truncated = False
    step_count = 0

    while not terminated and not truncated:
        mcts.root = Node(observation)
        mcts.run_mcts(num_simulations_per_step)
        action = mcts.get_best_action()

        if action is None:
            break

        next_observation, reward, terminated, truncated, info = env.step(action)
        total_episode_reward += reward
        observation = next_observation
        step_count += 1

    if reward > 0:
        successes += 1

    episode_rewards.append(total_episode_reward)

env.close()

success_rate = (successes / num_episodes) * 100
average_reward = np.mean(episode_rewards)

print("\n--- Evaluation Results ---")
print(f"Success Rate: {success_rate:.2f}%")
print(f"Average Reward per Episode: {average_reward:.4f}")